In [ ]:
from z3 import *

In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

from openai import OpenAI

In [ ]:
client = OpenAI(api_key="")

In [ ]:
def clean_code_snippet(snippet):
    """
    Clean a code snippet by removing triple backticks and language identifiers.
    
    Args:
        snippet (str): Raw code snippet
        
    Returns:
        str: Cleaned code snippet
    """
    # Remove triple backticks and language identifier
    cleaned = snippet.replace('```python\n', '').replace('```z3\n', '').replace('```', '')
    # Replace escaped newlines with actual newlines
    return cleaned.replace('\\n', '\n')

def write_snippets_to_file(iteration, snippets, filename="formatted_snippets.txt"):
    """
    Write formatted code snippets to a file.
    
    Args:
        iteration (int): Current iteration number
        snippets (dict): Dictionary containing the snippets
        filename (str): Name of the output file
    """
    mode = 'a' if iteration > 0 else 'w'  # Append if not first iteration
    with open(filename, 'a') as f:
        f.write(f"\nIteration {iteration}:\n")
        f.write("="*50 + "\n")
        
        for snippet_type, code in snippets.items():
            f.write(f"\n{snippet_type}:\n")
            f.write(clean_code_snippet(code))
            f.write("\n" + "-"*40 + "\n")

# Your existing code with modifications
results = {}
iteration = 0
with open("halu_q_a.txt", 'r') as f:
    for i in f:
        # Split knowledge, question, and answers
        knowledge_question = i.split("right_answer")[0]
        right_answer = i.split("right_answer")[1].split("hallucinated_answer")[0]
        hallucinated_answer = i.split("right_answer")[1].split("hallucinated_answer")[1]
        
        if iteration not in results:
            results[iteration] = {}
            
        # Your existing GPT-4 completions code remains the same
        completion_2 = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "user", 
                "content": (
                                        "Your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is the first example: "
                    "Here is the sentence: House of Anubis is a mystery television series developed for Nickelodeon based on the Dutch-Belgian television series '\Het Huis Anubis\'"
                    "Here is the conversion: "
                    "# Predicates "
                    "is_tv_series = Function('is_tv_series', StringSort(), BoolSort()) "
                    "developed_for = Function('developed_for', StringSort(), StringSort(), BoolSort()) "
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort()) "

                    "# Constants "
                    "house_of_anubis = StringVal('House of Anubis') "
                    "nickelodeon = StringVal('Nickelodeon') "
                    "het_huis_anubis = StringVal('Het Huis Anubis') "

                    "# Axioms "
                    "axioms = [ "
                    "    is_tv_series(house_of_anubis), "
                    "    developed_for(house_of_anubis, nickelodeon), "
                    "    based_on(house_of_anubis, het_huis_anubis) "
                    "]"
                    "Again, your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is the next example: "
                    "Here is the sentence: It first aired in September 2006 and the last episode was broadcast on December 4, 2009."
                    "Here is the conversion to Z3: " 
                    "# Predicates "
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), IntSort(), IntSort(), BoolSort()) "
                    "last_aired_on = Function('last_aired_in', StringSort(), IntSort(), IntSort(), IntSort(), BoolSort()) "

                    "# Constants "
                    "het_huis_anubis = StringVal('Het Huis Anubis') "

                    "# Axioms "
                    "axioms = [ "
                    "    first_aired_in(het_huis_anubis, 2006, 9, 1), "
                    "    last_aired_on(het_huis_anubis, 2009, 12, 4)  "
                    "]"
                    "Again, your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is the next example: "
                    "Here is the sentence: Question: The Dutch-Belgian television series that '\House of Anubis\' was based on first aired in what year? Answer: 2003"
                    "Here is the conversion to Z3: "
                    "# Predicates "
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort()) "
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), BoolSort()) "

                    "# Constants "
                    "house_of_anubis = StringVal('House of Anubis') "
                    "het_huis_anubis = StringVal('Het Huis Anubis') "

                    "# Axioms "
                    "axioms = [ "
                    "    based_on(house_of_anubis, het_huis_anubis), "
                    "    first_aired_in(het_huis_anubis, 2006) "
                    "]"
                    "# Query based on Answer"
                    "query = first_aired_in(het_huis_anubis, 2003)"
                    "Now convert this sentence to Z3 with a query based on the Answer, ONLY OUTPUT Z3 NOTHING ELSE: " + 
                    "Question: " + knowledge_question + " Answer: " + right_answer
                )
            }])
            
        results[iteration]["right_answer"] = completion_2.choices[0].message.content
        
        completion_3 = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "user", 
                "content": (
                                        "Your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is the first example: "
                    "Here is the sentence: House of Anubis is a mystery television series developed for Nickelodeon based on the Dutch-Belgian television series '\Het Huis Anubis\'"
                    "Here is the conversion: "
                    "# Predicates "
                    "is_tv_series = Function('is_tv_series', StringSort(), BoolSort()) "
                    "developed_for = Function('developed_for', StringSort(), StringSort(), BoolSort()) "
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort()) "

                    "# Constants "
                    "house_of_anubis = StringVal('House of Anubis') "
                    "nickelodeon = StringVal('Nickelodeon') "
                    "het_huis_anubis = StringVal('Het Huis Anubis') "

                    "# Axioms "
                    "axioms = [ "
                    "    is_tv_series(house_of_anubis), "
                    "    developed_for(house_of_anubis, nickelodeon), "
                    "    based_on(house_of_anubis, het_huis_anubis) "
                    "]"
                    "Again, your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is the next example: "
                    "Here is the sentence: It first aired in September 2006 and the last episode was broadcast on December 4, 2009."
                    "Here is the conversion to Z3: " 
                    "# Predicates "
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), IntSort(), IntSort(), BoolSort()) "
                    "last_aired_on = Function('last_aired_in', StringSort(), IntSort(), IntSort(), IntSort(), BoolSort()) "

                    "# Constants "
                    "het_huis_anubis = StringVal('Het Huis Anubis') "

                    "# Axioms "
                    "axioms = [ "
                    "    first_aired_in(het_huis_anubis, 2006, 9, 1), "
                    "    last_aired_on(het_huis_anubis, 2009, 12, 4)  "
                    "]"
                    "Again, your job is to act as a semantic parser converting a sentence I give you into Microsoft's Z3 logical prover syntax. "
                    "Here is the next example: "
                    "Here is the sentence: Question: The Dutch-Belgian television series that '\House of Anubis\' was based on first aired in what year? Answer: 2003"
                    "Here is the conversion to Z3: "
                    "# Predicates "
                    "based_on = Function('based_on', StringSort(), StringSort(), BoolSort()) "
                    "first_aired_in = Function('first_aired_in', StringSort(), IntSort(), BoolSort()) "

                    "# Constants "
                    "house_of_anubis = StringVal('House of Anubis') "
                    "het_huis_anubis = StringVal('Het Huis Anubis') "

                    "# Axioms "
                    "axioms = [ "
                    "    based_on(house_of_anubis, het_huis_anubis), "
                    "    first_aired_in(het_huis_anubis, 2006) "
                    "]"
                    "# Query based on Answer"
                    "query = first_aired_in(het_huis_anubis, 2003)"
                    "Now convert this sentence to Z3 with a query based on the Answer, ONLY OUTPUT Z3 NOTHING ELSE: " + 
                    "Question: " + knowledge_question + " Answer: " + hallucinated_answer
                )
            }])
            
        results[iteration]["hallucinated_answer"] = completion_3.choices[0].message.content
        
#         # Write the raw results to JSON
#         with open("results_raw.json", 'w') as file:
#             json.dump(results, file, indent=4)
        
        # Clean and write formatted snippets
        current_snippets = {
            "knowledge_question": knowledge_question,
            "right_answer": results[iteration]["right_answer"],
            "hallucinated_answer": results[iteration]["hallucinated_answer"]
        }
        write_snippets_to_file(iteration, current_snippets)
        
        #print(iteration)
        iteration += 1